# 로컬 개발 코드
- 로컬에서 주피터 노트북(Jupyter Notebook), 주피터 랩(JupyterLab) 또는 파이썬(Python)을 이용한다. 
- 사이킷 런(scikit-learn), 텐서플로우(tensorflow), 파이토치(pytorch)를 사용하여 딥러닝 프로그램을 개발한다.
- 파일명: 0_local_Object_Detection.ipynb

1. 데이터셋 준비(Data Setup)
- 로컬 저장소에서 전처리 및 학습에 필요한 학습 데이터셋을 준비한다.

2. 데이터 전처리(Data Preprocessing)
- 데이터셋의 분석 및 이미지 용량 축소(Image Compression), 라벨링 데이터 변환 등의 전처리를 수행한다.
- 데이터를 모델 학습에 사용할 수 있도록 가공한다.
- 추론과정에서 필요한 경우, 데이터 전처리에 사용된 객체를 meta_data 폴더 아래에 저장한다.

3. 학습 모델 훈련(Train Model)
- 데이터를 훈련에 사용할 수 있도록 가공한 뒤에 학습 모델을 구성한다. 
- 학습 모델을 준비된 데이터셋으로 훈련시킨다.
- 정확도(Accuracy)나 손실(Loss)등 학습 모델의 성능을 검증한다.
- 학습 모델의 성능 검증 후, 학습 모델을 배포한다.
- 배포할 학습 모델을 meta_data 폴더 아래에 저장한다.

4. 추론(Inference)
- 저장된 전처리 객체나 학습 모델 객체를 준비한다.
- 추론에 필요한 테스트 데이터셋을 준비한다.
- 배포된 학습 모델을 통해 테스트 데이터에 대한 추론을 진행한다. 

# 객체 탐지 (Object Detection) - YOLO v5
- 지금부터 이미지, 라벨링 데이터를 이용하여 객체 탐지를 진행해보고자 한다.

# 실시간 객체 인식을 통한 이륜차 위험요소 감지 서비스

### 사용할 데이터

![image](https://i.imgur.com/7Xz8d5M.gif)

- AI Hub에서 제공하는 Open Dataset인 [이륜자동차 안전 위험 시설물 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=572)와 Roboflow에서 제공하는 Open Dataset인 [Pothole Dataset](https://public.roboflow.com/object-detection/pothole)이라는 도로 데이터셋으로,

- 해당 데이터셋은 이미지 파일과 coco-json 형태의 파일들로 이루어져 있다.

- 이미지 데이터셋이 어떻게 json 형태로 boundingBox되어있는지 확인하고 이해한다.

- YOLO v5 모델 학습을 하기 위해서 Pytorch Yolo v5 txt 형태의 labeling 파일들로 변환한다.

### 사용할 7개의 도로 위험물 카테고리
- Pothole on road (포트홀)
- Person (사람)
- Garbage bag & sacks (쓰레기 봉지)
- Construction signs & Parking prohibited board (도로 사인 & 주차금지 표지)
- Traffic cone (트래픽콘)
- Filled pothole (메워진 포트홀)
- Manhole (맨홀)

In [10]:
# Imports
import os
import sys
from PIL import Image
import time
import numpy as np
import random
from pylabel import importer
import glob, json, os, shutil
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pprint import pprint
import glob, json
import torch
import utils
import cv2

## **1. 데이터셋 준비(Data Setup)**

In [5]:
file_list = os.listdir('./meta_data/Images/')
img_random=random.sample(file_list, 100)
img_random.
img_random

glob(r'./TrainingData/data/change_img/*.png')

AttributeError: 'list' object has no attribute 'save'

# 2. 데이터 전처리 

## 2-1) Image Compression  
---

In [ ]:
def change_img_qualty(original_path, change_path, qualty):
    """
    Change Image Qualty
    :param original_path: 원본 경로
    :param change_path: 변경 후 새롭게 저장될 경로
    :param qualty: Qualty(품질) 퍼센트(기본 : 85%)
    :return:
    """
    if not os.path.exists(change_path):
        os.mkdir(change_path)
    try:
        ims_list = os.listdir(original_path)
        ims_list.sort()
    except FileNotFoundError as e:
        print("이미지 원본 디렉터리가 존재하지 않습니다...")
        sys.exit(0)
    success_cnt = 0
    fail_cnt = 0
    for filename in ims_list:
        file = original_path + filename
        try:
            im = Image.open(file)
            im.save(os.path.join(change_path, filename), qualty=qualty)
            print("+ 성공 : {success}\n  "
                  "- {success_path}"
                  .format(success=file, success_path=os.path.join(change_path, filename))
                  )
            success_cnt += 1
        except Exception as e:
            print("+ 실패 : {fail}".format(fail=file))
            fail_cnt += 1
    print("\n성공 : {success_cnt} 건 / 실패 : {fail_cnt} 건".format(success_cnt=success_cnt, fail_cnt=fail_cnt))
    sys.exit(0)

In [ ]:
if __name__ == '__main__':
    original_path = './meta_data/Images/'
    change_path = './meta_data/change_img/'
    change_img_qualty(original_path, change_path,85)

## 2-1) Category Equalization
## 2-2) Convert COCO to YOLO
## 2-3) Convert json to txt

In [ ]:
# annotation 폴더 내의 json 파일 불러오기
LABEL_PATH = './meta_data/Annotations/'
IMAGE_PATH = './meta_data/Images/'
LABEL_SAVE_PATH = './meta_data/equalized_data/equalized_label'
IMAGE_SAVE_PATH = './meta_data/equalized_data/equalized_img'
json_list = os.listdir(LABEL_PATH)
img_list = os.listdir(IMAGE_PATH)

In [ ]:
# 카테고리
cat_1 = 0  # Animals(Dolls)                                    delete
cat_2 = 0  # Person
cat_3 = 0  # Garbage bag & sacks
cat_4 = 0  # Construction signs & Parking prohibited board
cat_5 = 0  # Traffic cone
cat_6 = 0  # Box                                               delete
cat_7 = 0  # Stones on road                                    delete
cat_8 = 0  # Pothole on road
cat_9 = 0  # Filled pothole
cat_10 = 0  # Manhole
cat_list = ["Animals(Dolls)", "Person", "Garbage bag & sacks", "Construction signs & Parking prohibited board",
            "Traffic cone",
            "Box", "Stones on road", "Pothole on road", "Filled pothole", "Manhole"]

In [ ]:
for file in json_list:
    # pylabel load
    dataset = importer.ImportCoco(LABEL_PATH + file)

    for i in range(len(dataset.df)):
        # if 라벨이 하나만 있을 때
        if len(dataset.df) == 1:
            # 이미지 복사를 위한 변수들
            cat_id = dataset.df.iloc[i].cat_id
            img_name = dataset.df.iloc[i].img_filename

            # txt 변경을 위한 변수들
            img_name2 = dataset.df.iloc[i].img_filename[:-3]
            bbox_x = dataset.df.iloc[i].ann_bbox_xmin
            bbox_y = dataset.df.iloc[i].ann_bbox_ymin
            height = dataset.df.iloc[i].ann_bbox_height
            width = dataset.df.iloc[i].ann_bbox_width
            img_height = dataset.df.iloc[i].img_height
            img_width = dataset.df.iloc[i].img_width
            img_category = int(dataset.df.iloc[i].cat_id) - 1

            if int(cat_id) == 2 and cat_2 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------

                # 카테고리 카운팅
                cat_2 += 1

            if int(cat_id) == 3 and cat_3 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------
                # 카테고리 카운팅
                cat_3 += 1
            if int(cat_id) == 4 and cat_4 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------
                # 카테고리 카운팅
                cat_4 += 1
            if int(cat_id) == 5 and cat_5 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------
                # 카테고리 카운팅
                cat_5 += 1
            if int(cat_id) == 8 and cat_8 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------
                # 카테고리 카운팅
                cat_8 += 1
            if int(cat_id) == 9 and cat_9 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------
                # 카테고리 카운팅
                cat_9 += 1
            if int(cat_id) == 10 and cat_10 <= 100:
                # ------------------- 이미지 복사 부분 -------------------
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(IMAGE_SAVE_PATH + cat_id):
                    os.makedirs(IMAGE_SAVE_PATH + cat_id)
                else:
                    pass
                # 파일을 새로운 경로로 복사한다.
                shutil.copy2(IMAGE_PATH + img_name, IMAGE_SAVE_PATH + cat_id + '/' + img_name)
                # ------------------------------------------------------

                # -------------------- 라벨 생성 부분 --------------------
                if img_height == 0:
                    img_height = 720
                else:
                    pass
                if img_width == 0:
                    img_width = 1280
                else:
                    pass

                dw = 1.0 / img_width
                dh = 1.0 / img_height

                x_center = bbox_x + width / 2.0
                y_center = bbox_y + height / 2.0

                x = x_center * dw
                y = y_center * dh
                w = width * dw
                h = height * dh
                # 폴더가 없으면 폴더 생성
                if not os.path.exists(LABEL_SAVE_PATH + cat_id):
                    os.makedirs(LABEL_SAVE_PATH + cat_id)
                else:
                    pass

                # 파일이 있으면
                if os.path.isfile(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt'):
                    pass
                # 파일이 없으면
                else:
                    f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'w')
                    f.close()

                f = open(LABEL_SAVE_PATH + cat_id + '/' + img_name2 + 'txt', 'a')
                f.write(str(img_category) + ' ')
                f.write(str(x) + ' ')
                f.write(str(y) + ' ')
                f.write(str(w) + ' ')
                f.write(str(h) + '\n')
                f.close()
                # ------------------------------------------------------
                # 카테고리 카운팅
                cat_10 += 1
        else:
            pass

## 라벨 카테고리 분포 시각화

In [ ]:
cat_count = [cat_1, cat_2, cat_3, cat_4, cat_5,
             cat_6, cat_7, cat_8, cat_9, cat_10]

plt.title("Category Status", size=20)
plt.bar(cat_list, cat_count, color=mcolors.TABLEAU_COLORS)
plt.xticks(rotation=90)
plt.show()

# 2. Create yaml
---

In [ ]:
# 이미지 파일 리스트 생성
train_img_list = glob(r'./TrainingData/data/change_img/*.png')
print(f'Image 갯수: {len(train_img_list)}')

In [ ]:
# train, validation 데이터 분리
train_img_list, val_img_list = train_test_split(train_img_list,
                                                  train_size=0.8,
                                                  random_state=42)

print(f'Train Image 갯수: {len(train_img_list)}')
print(f'Validation Image 갯수: {len(val_img_list)}')

In [ ]:
# train/val 이미지 경로 txt파일로 저장
with open(r'C:\Users\USER\T3Q_1_project\TrainingData\train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open(r'C:\Users\USER\T3Q_1_project\TrainingData\val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [ ]:
# ymal 파일 정보 확인
import yaml

with open(r'C:\Users\USER\T3Q_1_project\TrainingData\data.yaml', 'r') as f:
    data = yaml.full_load(f)

print(f'data.yaml 파일 정보: {data}')

data['train'] = r'C:\Users\USER\T3Q_1_project\TrainingData\train.txt'
data['val'] = r'C:\Users\USER\T3Q_1_project\TrainingData\val.txt'

with open(r'C:\Users\USER\T3Q_1_project\TrainingData\data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

## **3. 학습 모델 훈련 (Train Model) - YOLO v5**

객체 탐지를 위해 YOLO v6 모델을 Pytorch를 이용하여 구축하고자 한다.

In [ ]:
#!git clone https://github.com/knudatascientists/T3Q_projects_team_1/tree/main/yolo_v5/yolov5-master  # clone

In [ ]:
# %cd C://project/T3Q_projects_team_1/yolo_v5/yolov5-master

In [ ]:
#%pip install -qr requirements.txt 

In [ ]:
#!python train.py --batch 16 --epochs 50 --data C://Users/USER/T3Q_1_project/TrainingData/data.yaml  --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name T3Q_yolov5s_results

In [ ]:
# !python detect.py --weights C://Users/USER/T3Q_1_project/TrainingData/train_results/weights/best.pt --img 416 --conf 0.5 --source "https://www.youtube.com/watch?v=pIfB4sTGO-c"